#### Libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
from datetime import datetime, timedelta
import os
import requests
import threading
import msvcrt

#### Driver options

In [2]:
temp_folder = r"D:\Lostsale ĐM\download_tạm"

options = webdriver.ChromeOptions()
#options.add_argument('--headless')
options.add_experimental_option("prefs", {
    "download.default_directory": temp_folder,
    "safebrowsing.enabled": "false"
})
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(20)

#### Define functions

Clear_folder

In [3]:
def Clear_folder(folder_path=temp_folder):
    #folder_path = r'C:\PYTHON\SELENIUM\download'
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)

Login

In [4]:
def Login(username='208064', password='AAhZuuPq'):
    driver.get('https://pos.bachhoaxanh.com/')
    textbox_username = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/form/div[1]/input')
    textbox_password = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/form/div[2]/input')
    button_login = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/form/button')
    
    textbox_username.send_keys(username)
    textbox_password.send_keys(password)
    button_login.click()
    print('Đăng nhập thành công! \nUser_id:', username)

Get_to_the_webpage

In [5]:
def Get_to_the_webpage(report_name):
    report_link = 'https://pos.bachhoaxanh.com/reports/home/dashboard/'
    full_link = report_link + str(report_name)
    driver.get(full_link)
    time.sleep(5)
    driver.get(full_link)
    time.sleep(5)

Select_and_export_excel

In [6]:
def Select_and_export_excel(page_num, add_date=0, specific_day=None,  template=1):
    Get_to_the_webpage(page_num)
    
    # Chọn template đổ báo cáo
    report_template = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div[2]/div[2]/span/span/span/span')
    report_template.click()
    time.sleep(2)


    select_template = driver.find_element(By.XPATH, '/html/body/div[5]/div/div[2]/ul/li[' + str(1+template) + ']')
    select_template.click()
    time.sleep(45)

    # Chọn ngày từ - đến theo báo cáo
    if page_num in [827, 89]:
        date_location = 1
    else:
        date_location = 0

    # Xử lý nếu specific_day có giá trị
    if specific_day == None:
        specific_day = (datetime.now().date())
    else:
        specific_day = datetime.strptime(str(specific_day), '%Y%m%d').date()
    today = (datetime.now().date())
    sub = (specific_day - today).days

    date = (datetime.now().date() + timedelta(days=add_date + sub)).strftime('%d/%m/%Y')
    if page_num in [180]:
        pass
    else:
        from_date = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[1]/div['+ str(1 + date_location) +']/div[1]/div/span/span/input')
        from_date.send_keys(Keys.SHIFT + Keys.HOME)
        from_date.send_keys(Keys.BACKSPACE)
        from_date.send_keys(str(date) + ' 00:00')
        time.sleep(2)

    # Dùng try - except để tránh lỗi ở báo cáo 531
    try:
        to_date = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[1]/div['+ str(2 + date_location) +']/div[1]/div/span/span/input')
        to_date.send_keys(Keys.SHIFT + Keys.HOME)
        to_date.send_keys(Keys.BACKSPACE)
        to_date.send_keys(str(date) + ' 23:59')
        time.sleep(2)
    except NoSuchElementException:
        pass
    
    # Click vào vùng trống
    if page_num in [180]:
        pass
    else:
        from_date.click()

    # Nhấn button Xuất Excel
    button_export = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div[1]/button[2]')
    button_export.click()
    time.sleep(2)

Get_download_status

In [7]:
def Get_download_status():
    # Mở trang Download Manager và get elements cần thiết
    driver.get('https://pos.bachhoaxanh.com/reports/ManagerDownload')
    time.sleep(5)
    text_status = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div/div[2]/table/tbody/tr[1]/td[3]')
    status = text_status.text
    return status

Download_func

In [8]:
def Download_func(page_num, add_date, download_time=20, specific_day=None, folder_path_and_file_name=''):
    
    # Xử lý nếu specific_day có giá trị
    if specific_day == None:
        specific_day = (datetime.now().date())
    else:
        specific_day = datetime.strptime(str(specific_day), '%Y%m%d').date()
    today = (datetime.now().date())
    sub = (specific_day - today).days

    date = (datetime.now().date() + timedelta(days=add_date + sub)).strftime('%Y%m%d')
    file_dir = folder_path_and_file_name + date + '.xlsx'

    # Xóa hết file trong thư mục tạm và download file
    Clear_folder()
    time.sleep(2)
    button_download = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div/div[2]/table/tbody/tr[1]/td[4]/a')
    button_download.click()
    #time.sleep(download_time)

    # Chờ tối đa 200 giây đến khi tải xong file
    def check_excel_file_exists(folder_path=temp_folder):
        files = os.listdir(folder_path)
        for file in files:
            if file.endswith(".xlsx"):
                return True

    count = 0
    timeout = 200
    while count < timeout:
        if check_excel_file_exists() == True:
            time.sleep(3)
            #print('Đã tải xuống!')
            break
        else:
            time.sleep(1)
            count = count + 1

    # Đổi tên file và di chuyển đến thư mục đích
    folder_path = temp_folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('xlsx'):
            old_file_path = os.path.join(folder_path, file_name)
    new_file_path = file_dir

    # Đổi tên và di chuyển file
    os.rename(old_file_path, new_file_path)

Lets_download

In [9]:
def Lets_download():
    retry = True
    while retry:
        try:
            status = Get_download_status()
            retry = False
            time.sleep(10)
        except NoSuchElementException:
            status = Get_download_status()
            time.sleep(10)

File_dir

In [10]:
def file_dir(page_num, add_date, specific_day=None, folder_path_and_file_name=''):

    if specific_day == None:
        specific_day = (datetime.now().date())
    else:
        specific_day = datetime.strptime(str(specific_day), '%Y%m%d').date()
    today = (datetime.now().date())
    sub = (specific_day - today).days

    date = (datetime.now().date() + timedelta(days=add_date + sub)).strftime('%Y%m%d')
    file_dir = folder_path_and_file_name + date + '.xlsx'
    return file_dir

Check_file_exist

In [11]:
def Check_file_exist(file_dir=file_dir):   
    if os.path.exists(file_dir):
        go_on = False
    else:
        go_on = True
    return go_on

Main

In [12]:
def Main(page_num, add_date, download_time, specific_day=None, folder_path_and_file_name='', template=1):
    a = file_dir(page_num, add_date, specific_day, folder_path_and_file_name)
    print('Preparing for downloading:', a)
    go_on = Check_file_exist(file_dir=a)

    retry = True
    while retry:
        if go_on == True:
            # Chọn điều kiện đổ báo cáo
            print('[---START---]')
            print('Selecting report template...')
            Select_and_export_excel(page_num, add_date, specific_day, template)
            print('Exporting excel file...')

            # Mở trang download
            print('Going to Download Manager...')
            Lets_download()

            # Kiểm tra trạng thái và 
            status_success = 'Đã xuất xong'
            status_fail_1 = 'Lỗi xuất excel'
            status_fail_2 = 'Lỗi lấy dữ liệu'

            print('[---Wait a damn minute---]')
            checking_mode = True
            while checking_mode:

                status = Get_download_status()

                if status_success in status:
                    print(status)
                    Download_func(page_num, add_date, download_time, specific_day, folder_path_and_file_name)
                    print('Successfully!')
                    checking_mode = False

                elif (status_fail_1 in status) or (status_fail_2 in status):
                    print(status)
                    print('[---Download fail---]')
                    time.sleep(1)
                    print('Try again, >_<')
                    time.sleep(1)
                    print('Selecting report template...')
                    Select_and_export_excel(page_num, add_date, specific_day, template)
                    print('Exporting excel file...')

                else:
                    print(status)
                    time.sleep(30)
                    
            retry = False

        else:
            print('This file already exists!')
            print('Do you want to REPLACE it, Y/N?')
            time.sleep(0.5)
            isReplace = input('Do you want to REPLACE it, Y/N?')
            isReplace = isReplace.upper()
            if isReplace == 'Y':
                go_on = True
                os.remove(a)
                print('File has been removed!')
            else:
                print('Cancelled!')
                break

#### Login

In [13]:
Login()

Đăng nhập thành công! 
User_id: 208064


#### Download data

In [14]:
today = datetime.now().date()
from_date = today - timedelta(days=7)
to_date = today 
from_date = int(from_date.strftime('%Y%m%d'))
to_date = int(to_date.strftime('%Y%m%d'))

In [15]:
# path = 'D:/Lostsale ĐM/587/587_downloads/DATA-3191-'
# Main(page_num=3191, add_date=0, download_time=60, specific_day=None, folder_path_and_file_name=path)

In [16]:

path = 'D:/Lostsale ĐM/Facing/facing_downloads/DATA-180-'
Main(page_num=180, add_date=0, download_time=60, specific_day=None, folder_path_and_file_name=path)

Preparing for downloading: D:/Lostsale ĐM/Facing/facing_downloads/DATA-180-20231116.xlsx
[---START---]
Selecting report template...


Exporting excel file...
Going to Download Manager...
[---Wait a damn minute---]
11% loading
41% loading
72% loading
97% loading
Đã xuất xong, có thể tải file
Successfully!


In [17]:
path = 'D:/Lostsale ĐM/Candate/Candate downloads/DATA-603-'
Main(page_num=603, add_date=-1, download_time=60, specific_day=None, folder_path_and_file_name=path)

Preparing for downloading: D:/Lostsale ĐM/Candate/Candate downloads/DATA-603-20231115.xlsx
[---START---]
Selecting report template...
Exporting excel file...
Going to Download Manager...
[---Wait a damn minute---]
2% loading
16% loading
30% loading
44% loading
59% loading
73% loading
89% loading
Đã xuất xong, có thể tải file
Successfully!


In [18]:

path = 'D:/Lostsale ĐM/Điểm NCC/Điểm NCC downloads/DATA-273-'
Main(page_num=273, add_date=0, download_time=60, specific_day=None, folder_path_and_file_name=path)

Preparing for downloading: D:/Lostsale ĐM/Điểm NCC/Điểm NCC downloads/DATA-273-20231116.xlsx
[---START---]
Selecting report template...


Exporting excel file...
Going to Download Manager...
[---Wait a damn minute---]
Đã xuất xong, có thể tải file
Successfully!


In [19]:

path = 'D:/Lostsale ĐM/PX/PX downloads/DATA-72-'
Main(page_num=72, add_date=-1, download_time=60, specific_day=None, folder_path_and_file_name=path)

Preparing for downloading: D:/Lostsale ĐM/PX/PX downloads/DATA-72-20231115.xlsx
[---START---]
Selecting report template...
Exporting excel file...
Going to Download Manager...
[---Wait a damn minute---]
4% loading
18% loading
32% loading
48% loading
65% loading
81% loading
97% loading
97% loading
Đã xuất xong, có thể tải file
Successfully!


In [20]:

path = 'D:/Lostsale ĐM/PN/PN downloads/DATA-951-'
Main(page_num=951, add_date=-1, download_time=60, specific_day=None, folder_path_and_file_name=path)

Preparing for downloading: D:/Lostsale ĐM/PN/PN downloads/DATA-951-20231115.xlsx
[---START---]
Selecting report template...
Exporting excel file...
Going to Download Manager...
[---Wait a damn minute---]
Hiện tại chưa tới lượt đổ báo cáo của bạn, vui lòng chờ trong ít phút !
Hiện tại chưa tới lượt đổ báo cáo của bạn, vui lòng chờ trong ít phút !
Đã xuất xong, có thể tải file
Successfully!


In [21]:
# for date in range(from_date, to_date):
#     path = 'D:/Lostsale ĐM/89/89dl/DATA-89-'
#     Main(page_num=89, add_date=0, download_time=60, specific_day=date, folder_path_and_file_name=path)

In [22]:
path = 'D:/Lostsale ĐM/89/89dl/DATA-89-'
Main(page_num=89, add_date=-1, download_time=60, specific_day=None, folder_path_and_file_name=path)

Preparing for downloading: D:/Lostsale ĐM/89/89dl/DATA-89-20231115.xlsx
[---START---]
Selecting report template...
Exporting excel file...
Going to Download Manager...
[---Wait a damn minute---]
Đã xuất xong, có thể tải file
Successfully!


In [23]:

path = 'D:/Lostsale ĐM/911/911 downloads/DATA-808-'
Main(page_num=808, add_date=-1, download_time=60, specific_day=None, folder_path_and_file_name=path)

Preparing for downloading: D:/Lostsale ĐM/911/911 downloads/DATA-808-20231115.xlsx
[---START---]
Selecting report template...
Exporting excel file...
Going to Download Manager...
[---Wait a damn minute---]
Hiện tại chưa tới lượt đổ báo cáo của bạn, vui lòng chờ trong ít phút !
Hiện tại chưa tới lượt đổ báo cáo của bạn, vui lòng chờ trong ít phút !
Đã xuất xong, có thể tải file
Successfully!


In [24]:
# for date in range(20231101, 20231113):
#     path = 'D:/Lostsale ĐM/308/308dl/DATA-308-'
#     Main(page_num=308, add_date=0, download_time=60, specific_day=date, folder_path_and_file_name=path)

In [25]:


path = 'D:/Lostsale ĐM/554/554 downloads/DATA-554-'
Main(page_num=554, add_date=-1, download_time=60, specific_day=None, folder_path_and_file_name=path)

Preparing for downloading: D:/Lostsale ĐM/554/554 downloads/DATA-554-20231115.xlsx
[---START---]
Selecting report template...
Exporting excel file...
Going to Download Manager...
[---Wait a damn minute---]
Hiện tại chưa tới lượt đổ báo cáo của bạn, vui lòng chờ trong ít phút !
Hiện tại chưa tới lượt đổ báo cáo của bạn, vui lòng chờ trong ít phút !
Hiện tại chưa tới lượt đổ báo cáo của bạn, vui lòng chờ trong ít phút !
Đã xuất xong, có thể tải file
Successfully!
